In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection
import os

# ================= 配置区域 =================

# 在 .ipynb 文件中，直接使用 os.getcwd() 即可
CURRENT_DIR = os.getcwd()
print(f"当前工作目录: {CURRENT_DIR}")

# 1. 输入文件 (上一步生成的文件)
INPUT_FILE = os.path.join(CURRENT_DIR, "question_expanded_result_v2.xlsx")

# 2. 输出文件
OUTPUT_FILE = os.path.join(CURRENT_DIR, "retrieval_comparison_result.xlsx")

# 3. Milvus 配置
MILVUS_HOST = "192.168.16.138"
MILVUS_PORT = "19530"
COLLECTION_NAME = "report_analysis1"  
VECTOR_FIELD = "vector"
OUTPUT_FIELDS = ["text_content"]      

# 4. 模型配置
MODEL_NAME = 'all-mpnet-base-v2'

# ===========================================

def search_milvus(collection, model, query_text, top_k=3):
    """
    输入文本，返回召回的文本列表
    """
    if not query_text or pd.isna(query_text) or str(query_text).strip() == "":
        return []

    # 1. 生成向量
    query_vector = model.encode([str(query_text)])

    # 2. 搜索参数
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 10}
    }

    # 3. 执行搜索
    try:
        results = collection.search(
            data=query_vector, 
            anns_field=VECTOR_FIELD, 
            param=search_params, 
            limit=top_k, 
            output_fields=OUTPUT_FIELDS
        )
        
        # 4. 解析结果
        hits = results[0]
        retrieved_items = []
        for hit in hits:
            # 获取文本内容
            content = hit.entity.get("text_content")
            score = hit.distance 
            # 格式化输出：分数 + 内容摘要
            retrieved_items.append(f"【距离:{score:.4f}】{content[:150]}...") 
            
        return retrieved_items

    except Exception as e:
        print(f"检索出错: {e}")
        return [f"Error: {e}"]

def main():
    print("--- 开始检索对比任务 ---")

    # 1. 连接 Milvus
    print(f"正在连接 Milvus ({MILVUS_HOST})...")
    try:
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
        if not utility.has_collection(COLLECTION_NAME): # 增加一个检查
             print(f"❌ 错误：集合 '{COLLECTION_NAME}' 不存在，请先运行入库脚本！")
             return
             
        collection = Collection(COLLECTION_NAME)
        collection.load() 
        print(f"集合 '{COLLECTION_NAME}' 加载成功，当前数据量: {collection.num_entities}")
    except Exception as e:
        print(f"❌ 连接 Milvus 失败: {e}")
        # 这里为了防止还没连上就报错，通常建议检查网络，但在notebook里为了调试方便，可以先继续或return
        return

    # 2. 加载模型
    print(f"正在加载模型 '{MODEL_NAME}'...")
    model = SentenceTransformer(MODEL_NAME)

    # 3. 读取 Excel
    print(f"正在读取文件: {INPUT_FILE}")
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 文件不存在: {INPUT_FILE}")
        print(f"请确认文件是否在: {CURRENT_DIR}")
        return
    
    try:
        df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"❌ 读取Excel失败: {e}")
        return
    
    # 检查列名
    required_cols = ['方案1_单句扩写', '方案2_潜在追问']
    for col in required_cols:
        if col not in df.columns:
            print(f"❌ Excel 中缺少列: {col}，请检查上一步生成的文件。")
            print(f"当前列名: {list(df.columns)}")
            return

    # 准备存储结果
    scheme1_results = []
    scheme2_results = []

    total = len(df)
    print(f"开始处理 {total} 条查询...")

    # 4. 循环检索
    for index, row in df.iterrows():
        # 简单的进度打印
        if (index + 1) % 5 == 0:
            print(f"[{index+1}/{total}] 正在检索...")
        
        # --- 方案 1 检索 ---
        q1 = row['方案1_单句扩写']
        res1 = search_milvus(collection, model, q1, top_k=3)
        scheme1_results.append("\n\n".join(res1))

        # --- 方案 2 检索 ---
        q2 = row['方案2_潜在追问']
        res2 = search_milvus(collection, model, q2, top_k=3)
        scheme2_results.append("\n\n".join(res2))

    # 5. 保存结果
    df['方案1_召回切片(Top3)'] = scheme1_results
    df['方案2_召回切片(Top3)'] = scheme2_results

    print(f"正在保存对比结果到: {OUTPUT_FILE}")
    try:
        df.to_excel(OUTPUT_FILE, index=False)
        print("✅ 处理完成！请打开 Excel 查看两种方案的召回效果差异。")
    except PermissionError:
        print("❌ 保存失败：请关闭正在打开的 Excel 文件后重试。")

# 在 Jupyter 中直接运行
if __name__ == "__main__":
    # 引入 utility 避免 undefined 错误
    from pymilvus import utility 
    main()

d:\jupyter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


当前工作目录: d:\jupyter
--- 开始检索对比任务 ---
正在连接 Milvus (192.168.16.138)...
集合 'report_analysis1' 加载成功，当前数据量: 671
正在加载模型 'all-mpnet-base-v2'...
正在读取文件: d:\jupyter\question_expanded_result_v2.xlsx
开始处理 83 条查询...
[5/83] 正在检索...
[10/83] 正在检索...
[15/83] 正在检索...
[20/83] 正在检索...
[25/83] 正在检索...
[30/83] 正在检索...
[35/83] 正在检索...
[40/83] 正在检索...
[45/83] 正在检索...
[50/83] 正在检索...
[55/83] 正在检索...
[60/83] 正在检索...
[65/83] 正在检索...
[70/83] 正在检索...
[75/83] 正在检索...
[80/83] 正在检索...
正在保存对比结果到: d:\jupyter\retrieval_comparison_result.xlsx
✅ 处理完成！请打开 Excel 查看两种方案的召回效果差异。
